In [1]:
# Import packages

# For data manipulation

import numpy as np
import pandas as pd

# For data visualization

import matplotlib.pyplot as plt
import seaborn as sns

# For displaying all of the columns in dataframes

pd. set_option('display-max_columns', None)

# For data modeling

from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn. linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# For metrics and helpful functions

from sklearn.model_selection import GridSearchCV, t r a i n _ t e s t _ s p l i t
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn-metrics import roc_auc_score, roc_curve
from sklearn. tree import plot_tree

# For saving models

import pickle

SyntaxError: invalid syntax (4169524430.py, line 28)